# Group Members

## Caleb Miller 
### cm3962@drexel.edu

## Hashim Afzal
### ha695@drexel.edu

In [188]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# - !wget https://mirror.jframeworks.com/apache/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!wget https://dlcdn.apache.org/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz
#    https://spark.apache.org/downloads.html -> click on Download Spark: spark-3.5.4-bin-hadoop3.tgz
!tar xvf spark-3.5.5-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark==3.5.5
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.5.5-bin-hadoop3"
#- Run the followings:

--2025-03-13 03:20:18--  https://dlcdn.apache.org/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400724056 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.5-bin-hadoop3.tgz.16’

spark-3.5.5-bin-had 100%[===================>] 382.16M   138MB/s    in 2.8s    

2025-03-13 03:20:21 (138 MB/s) - ‘spark-3.5.5-bin-hadoop3.tgz.16’ saved [400724056/400724056]

spark-3.5.5-bin-hadoop3/
spark-3.5.5-bin-hadoop3/jars/
spark-3.5.5-bin-hadoop3/jars/HikariCP-2.5.1.jar
spark-3.5.5-bin-hadoop3/jars/JLargeArrays-1.5.jar
spark-3.5.5-bin-hadoop3/jars/JTransforms-3.1.jar
spark-3.5.5-bin-hadoop3/jars/RoaringBitmap-0.9.45.jar
spark-3.5.5-bin-hadoop3/jars/ST4-4.0.4.jar
spark-3.5.5-bin-hadoop3/jars/activation-1.1.1.jar
spark-3.5.5-bin-hadoop3/jars/aircompressor-0.27.jar
spark-3.5.5-bin-hadoop3/jar

In [189]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## HW 3

In this assignment, we will learn about locality sensitive hashing in Apache Spark.

Start by running the code below if you are using Google Colab

In [190]:
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q https://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
# !tar xf spark-2.4.5-bin-hadoop2.7.tgz

In [191]:
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [192]:
# !pip install -q findspark
# !pip install pyspark

In [193]:
from pyspark.sql import SparkSession

In [194]:
APP_NAME = "HW3"

In [195]:
spark = SparkSession.builder.appName(APP_NAME).getOrCreate()

In [196]:
spark

1. Load the wikipedia links dataset provided in this assignment as a dataframe.

In [197]:
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","
file_type = "csv"

df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter).csv("/content/drive/MyDrive/HW3/wikipedia_links.csv")

In [198]:
df.printSchema()

root
 |-- link: string (nullable = true)



In [199]:
df.show(5, truncate=False)

+--------------------------------------------------------+
|link                                                    |
+--------------------------------------------------------+
|http://en.wikipedia.org/wiki/Lungotevere_dei_Mellini    |
|http://en.wikipedia.org/wiki/South_Rowan_High_School    |
|http://en.wikipedia.org/wiki/Haikou_College_of_Economics|
|http://en.wikipedia.org/wiki/Saylesville,_Rhode_Island  |
|http://en.wikipedia.org/wiki/Wallington,_New_Jersey     |
+--------------------------------------------------------+
only showing top 5 rows



2. We would like to keep the name of the Wikipedia entry only. Parse the links on the '/' symbol and keep only the last part (you may use PySpark functions for string splitting).

In [200]:
from pyspark.sql.functions import split, col

df = df.withColumn("wiki_entry", split(col("link"), "/").getItem(4))
df = df.drop("link")
df.show(5, truncate=False)


+---------------------------+
|wiki_entry                 |
+---------------------------+
|Lungotevere_dei_Mellini    |
|South_Rowan_High_School    |
|Haikou_College_of_Economics|
|Saylesville,_Rhode_Island  |
|Wallington,_New_Jersey     |
+---------------------------+
only showing top 5 rows



3. Add an ID column that will be a sequential integer between 1 and len(wiki) in our Wikipedia links table. You may order the column using any other column of your choice.

In [201]:
df.createOrReplaceTempView('wiki_links')
df = spark.sql('select row_number() over (order by "wiki_entry") as ID, * from wiki_links')
df.show(5, truncate=False)

+---+---------------------------+
|ID |wiki_entry                 |
+---+---------------------------+
|1  |Lungotevere_dei_Mellini    |
|2  |South_Rowan_High_School    |
|3  |Haikou_College_of_Economics|
|4  |Saylesville,_Rhode_Island  |
|5  |Wallington,_New_Jersey     |
+---+---------------------------+
only showing top 5 rows



In [202]:
from pyspark.sql.functions import min, max

In [203]:
df.select(min("ID"), max("ID")).show()

+-------+-------+
|min(ID)|max(ID)|
+-------+-------+
|      1|  60000|
+-------+-------+



4. Clean up the data by removing missing data.

In [204]:
df = df.dropna()

5. Tokenize the words by splitting on the underscore symbol, count vectorize using a vocab size of 1,000,000, and create feature vectors using the count vectors. Your final result should be a dataframe containing the ID we previously generated and the feature vectors.

In [205]:
from pyspark.ml.feature import RegexTokenizer
from pyspark.ml.feature import MinHashLSH
from pyspark.sql.types import *
from pyspark.ml.feature import Tokenizer, CountVectorizer
from pyspark.sql.functions import udf, col

regexTokenizer = RegexTokenizer(inputCol="wiki_entry", outputCol="words", pattern="_")
countTokens = udf(lambda words: len(words), IntegerType())
regexTokenized = regexTokenizer.transform(df)


In [206]:
regexTokenized.select("ID", "words") \
  .withColumn("tokens", countTokens(col("words"))) \
    .show(truncate=False)

+---+-------------------------------------------+------+
|ID |words                                      |tokens|
+---+-------------------------------------------+------+
|1  |[lungotevere, dei, mellini]                |3     |
|2  |[south, rowan, high, school]               |4     |
|3  |[haikou, college, of, economics]           |4     |
|4  |[saylesville,, rhode, island]              |3     |
|5  |[wallington,, new, jersey]                 |3     |
|6  |[saint, andrew's, chapel]                  |3     |
|7  |[nemperor, records]                        |2     |
|8  |[kepler-37b]                               |1     |
|9  |[ecomuseo, della, montagna, pistoiese]     |4     |
|10 |[virtual, network, computing]              |3     |
|11 |[newman, school]                           |2     |
|12 |[it's, christmas, again]                   |3     |
|13 |[band, expander]                           |2     |
|14 |[beehive, house]                           |2     |
|15 |[bunding]                 

In [207]:
vocabSize = 1000000
cvModel = CountVectorizer().setInputCol("words").setOutputCol("features").setVocabSize(vocabSize).setMinDF(10).fit(regexTokenized)

In [208]:
from pyspark.sql.functions import udf
from pyspark.ml.linalg import SparseVector
from pyspark.sql.types import BooleanType

isNoneZeroVector = udf(lambda v: v.numNonzeros() > 0, BooleanType())
vectorizedDF = cvModel.transform(regexTokenized).filter(isNoneZeroVector(col("features"))).select(col("ID"), col("features"))
vectorizedDF.show(20, truncate=False)

+---+----------------------------------------------------------------------------+
|ID |features                                                                    |
+---+----------------------------------------------------------------------------+
|2  |(2171,[6,16,29],[1.0,1.0,1.0])                                              |
|3  |(2171,[0,37],[1.0,1.0])                                                     |
|4  |(2171,[90,1641],[1.0,1.0])                                                  |
|5  |(2171,[8,426],[1.0,1.0])                                                    |
|6  |(2171,[360],[1.0])                                                          |
|7  |(2171,[298],[1.0])                                                          |
|10 |(2171,[339,1180],[1.0,1.0])                                                 |
|11 |(2171,[6],[1.0])                                                            |
|12 |(2171,[1031,1115,1517],[1.0,1.0,1.0])                                       |
|13 

In [209]:
vectorizedDF.filter(~isNoneZeroVector(col("features"))).show()

+---+--------+
| ID|features|
+---+--------+
+---+--------+



6. In the last part of the assignment, use the MinHashLSH model to fit our feature vectors and then find all pairs using a threshold of 0.3.

In [210]:
mh = MinHashLSH(inputCol="features", outputCol="hashes", seed=1)
model = mh.fit(vectorizedDF)
model.transform(vectorizedDF).show(truncate=False)

+---+----------------------------------------------------------------------------+-----------------+
|ID |features                                                                    |hashes           |
+---+----------------------------------------------------------------------------+-----------------+
|2  |(2171,[6,16,29],[1.0,1.0,1.0])                                              |[[4.03125691E8]] |
|3  |(2171,[0,37],[1.0,1.0])                                                     |[[7.54458509E8]] |
|4  |(2171,[90,1641],[1.0,1.0])                                                  |[[3.79490304E8]] |
|5  |(2171,[8,426],[1.0,1.0])                                                    |[[7.5182261E7]]  |
|6  |(2171,[360],[1.0])                                                          |[[2.3816998E8]]  |
|7  |(2171,[298],[1.0])                                                          |[[7.53720512E8]] |
|10 |(2171,[339,1180],[1.0,1.0])                                                 |[[1.24317

In [211]:
vectorizedDF.describe().show()

+-------+------------------+
|summary|                ID|
+-------+------------------+
|  count|             40402|
|   mean|30029.894534924013|
| stddev| 17347.47954078778|
|    min|                 2|
|    max|             59997|
+-------+------------------+



In [212]:
size_of_whole_df = vectorizedDF.count()
print(f"Number of rows: {size_of_whole_df}")

Number of rows: 40402


NOTE: WE APPLIED .LIMIT TO HOLD THE ENTIRE DATASET DUE TO A BUG

FOR SOME REASON WE KEPT GETTING AN ERROR ABOUT NONZERO OUTPUT

ERROR MESSAGE:

`Caused by: java.lang.IllegalArgumentException: requirement failed: Must have at least 1 non zero entry.`

## Per our email thread with Professor Toutounchian on Wednesday, March 12 to Thursday, March 13:

### "It seems would be a bug but honestly do not know the reason of error in the newer version of Pyspaark. So, your work is fine."

In [213]:
DF = vectorizedDF.limit(size_of_whole_df)  # For some reason, this will only work if we .limit the df to the ENTIRE dataset -- not sure why seems like a bug
threshold = 0.3
model.approxSimilarityJoin(DF, DF, threshold).filter("distCol != 0").show()


+--------------------+--------------------+-------------------+
|            datasetA|            datasetB|            distCol|
+--------------------+--------------------+-------------------+
|{2, (2171,[6,16,2...|{52852, (2171,[6,...|               0.25|
|{2, (2171,[6,16,2...|{36993, (2171,[6,...|               0.25|
|{2, (2171,[6,16,2...|{24757, (2171,[6,...|               0.25|
|{110, (2171,[30,5...|{20110, (2171,[30...| 0.2222222222222222|
|{194, (2171,[101,...|{31855, (2171,[10...|               0.25|
|{209, (2171,[66,3...|{20209, (2171,[66...|0.19999999999999996|
|{254, (2171,[0,1,...|{20254, (2171,[0,...| 0.2857142857142857|
|{264, (2171,[1,18...|{44468, (2171,[1,...|               0.25|
|{286, (2171,[23,3...|{44118, (2171,[23...|               0.25|
|{289, (2171,[1,34...|{19825, (2171,[1,...|0.16666666666666663|
|{289, (2171,[1,34...|{12254, (2171,[1,...|0.16666666666666663|
|{350, (2171,[20,4...|{20350, (2171,[20...|0.16666666666666663|
|{390, (2171,[0,1,...|{49279, (2171,[0,.

In [214]:
DF.show()

+---+--------------------+
| ID|            features|
+---+--------------------+
|  2|(2171,[6,16,29],[...|
|  3|(2171,[0,37],[1.0...|
|  4|(2171,[90,1641],[...|
|  5|(2171,[8,426],[1....|
|  6|  (2171,[360],[1.0])|
|  7|  (2171,[298],[1.0])|
| 10|(2171,[339,1180],...|
| 11|    (2171,[6],[1.0])|
| 12|(2171,[1031,1115,...|
| 13|  (2171,[435],[1.0])|
| 14|   (2171,[25],[1.0])|
| 17|(2171,[22,51,82,7...|
| 19|(2171,[236,344,35...|
| 25|  (2171,[314],[1.0])|
| 26|  (2171,[306],[1.0])|
| 27|(2171,[1,32,1214]...|
| 28|(2171,[0,1,6,18,5...|
| 29|(2171,[3,604],[1....|
| 30|(2171,[36,130],[1...|
| 31|(2171,[0,57],[1.0...|
+---+--------------------+
only showing top 20 rows



# FOR REFERENCE

## This is the code that was breaking for us -- same code except this didn't apply .limit(size_of_whole_df)

## Per our email thread with Professor Toutounchian on Wednesday, March 12 to Thursday, March 13:

### "It seems would be a bug but honestly do not know the reason of error in the newer version of Pyspaark. So, your work is fine."

In [215]:
threshold = 0.3
model.approxSimilarityJoin(vectorizedDF, vectorizedDF, threshold).filter("distCol != 0").show()

Py4JJavaError: An error occurred while calling o2079.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 425.0 failed 1 times, most recent failure: Lost task 0.0 in stage 425.0 (TID 260) (67cbc8cc9838 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`LSHModel$$Lambda$4183/377444275`: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => array<struct<type:tinyint,size:int,indices:array<int>,values:array<double>>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage4.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.jav